In [1]:
import os
import warnings

import seaborn as sns
from absl import app
from absl import flags
import matplotlib
matplotlib.use('Agg')
from matplotlib import figure  # pylint: disable=g-import-not-at-top
from matplotlib.backends import backend_agg
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp



tf.enable_v2_behavior()
warnings.simplefilter(action='ignore')

tfd = tfp.distributions

In [2]:
import h5py
filename = "/Volumes/Suyang_T5/Paulsson_lab/Dyes/20210812/barcodes_normalized.h5"
with h5py.File(filename, "r") as f:
    BFP_normed = f.get('BFP_data')[:]
    BFP_label = f.get('BFP_label')[:]  # BFP RFP label second column for the other color
    RFP_normed = f.get('RFP_data')[:]
    RFP_label = f.get('RFP_label')[:]
    all_normed = f.get('all_data')[:]
    all_label = f.get('all_label')[:]


In [3]:
# reshape all data to be channel last

BFP_normed = BFP_normed.reshape((BFP_normed.shape[0], 52, 24,2))
RFP_normed = RFP_normed.reshape((RFP_normed.shape[0], 52, 24,2))
all_normed = all_normed.reshape((all_normed.shape[0], 52, 24,2))

# BFP_label = BFP_label.reshape((BFP_label.shape[0], ))
# RFP_label = RFP_label.reshape((RFP_label.shape[0], ))
all_label = all_label.reshape((all_label.shape[0], ))

In [7]:

IMAGE_SHAPE = [52, 24, 2]
# NUM_TRAIN_EXAMPLES = 60000
# NUM_HELDOUT_EXAMPLES = 10000
NUM_CLASSES = 4

learning_rate = 0.001

num_epochs=20
batch_size=128
data_dir = os.path.join("output","bayesian_neural_network/data")
# Directory to put the model's fit.
model_dir = os.path.join("output","bayesian_neural_network/")
viz_steps = 400
num_monte_carlo=50
fake_data=False



def plot_weight_posteriors(names, qm_vals, qs_vals, fname):
  """Save a PNG plot with histograms of weight means and stddevs.

  Args:
    names: A Python `iterable` of `str` variable names.
      qm_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior means of weight varibles.
    qs_vals: A Python `iterable`, the same length as `names`,
      whose elements are Numpy `array`s, of any shape, containing
      posterior standard deviations of weight varibles.
    fname: Python `str` filename to save the plot to.
  """
  fig = figure.Figure(figsize=(6, 3))
  canvas = backend_agg.FigureCanvasAgg(fig)

  ax = fig.add_subplot(1, 2, 1)
  for n, qm in zip(names, qm_vals):
    sns.distplot(qm.reshape([-1]), ax=ax, label=n)
  ax.set_title('weight means')
  ax.set_xlim([-1.5, 1.5])
  ax.legend()

  ax = fig.add_subplot(1, 2, 2)
  for n, qs in zip(names, qs_vals):
    sns.distplot(qs.reshape([-1]), ax=ax)
  ax.set_title('weight stddevs')
  ax.set_xlim([0, 1.])

  fig.tight_layout()
#   canvas.draw()
  canvas.print_figure(fname, format='png')
  print('saved {}'.format(fname))


def plot_heldout_prediction(input_vals, probs,
                            fname, n=10, title=''):
  """Save a PNG plot visualizing posterior uncertainty on heldout data.

  Args:
    input_vals: A `float`-like Numpy `array` of shape
      `[num_heldout] + IMAGE_SHAPE`, containing heldout input images.
    probs: A `float`-like Numpy array of shape `[num_monte_carlo,
      num_heldout, num_classes]` containing Monte Carlo samples of
      class probabilities for each heldout sample.
    fname: Python `str` filename to save the plot to.
    n: Python `int` number of datapoints to vizualize.
    title: Python `str` title for the plot.
  """
  fig = figure.Figure(figsize=(9, 3*n))
  canvas = backend_agg.FigureCanvasAgg(fig)
  for i in range(n):
    ax = fig.add_subplot(n, 3, 3*i + 1)
    ax.imshow(input_vals[i, :].reshape(IMAGE_SHAPE[:-1]), interpolation='None')

    ax = fig.add_subplot(n, 3, 3*i + 2)
    for prob_sample in probs:
      sns.barplot(np.arange(10), prob_sample[i, :], alpha=0.1, ax=ax)
      ax.set_ylim([0, 1])
    ax.set_title('posterior samples')

    ax = fig.add_subplot(n, 3, 3*i + 3)
    sns.barplot(np.arange(10), np.mean(probs[:, i, :], axis=0), ax=ax)
    ax.set_ylim([0, 1])
    ax.set_title('predictive probs')
  fig.suptitle(title)
  fig.tight_layout()
#   canvas.draw()
  canvas.print_figure(fname, format='png')
  print('saved {}'.format(fname))


def create_model():
  """Creates a Keras model using the LeNet-5 architecture.

  Returns:
      model: Compiled Keras model.
  """
  # KL divergence weighted by the number of training samples, using
  # lambda function to pass as input to the kernel_divergence_fn on
  # flipout layers.
  kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) /  # pylint: disable=g-long-lambda
                            tf.cast(NUM_TRAIN_EXAMPLES, dtype=tf.float32))

  # Define a LeNet-5 model using three convolutional (with max pooling)
  # and two fully connected dense layers. We use the Flipout
  # Monte Carlo estimator for these layers, which enables lower variance
  # stochastic gradients than naive reparameterization.

#   model.add(Conv2D(32,
#                  activation='relu',
#                  input_shape=input_shape,
#                  kernel_size=(3, 3)))
# model.add(Conv2D(64, activation='relu',
#                  kernel_size=(3, 3)))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.35))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))  

  model = tf.keras.models.Sequential([
  tfp.layers.Convolution2DFlipout(
      32, kernel_size=3, padding='SAME',
      kernel_divergence_fn=kl_divergence_function,
      activation=tf.nn.relu),
  tfp.layers.Convolution2DFlipout(
      64, kernel_size=3,
      activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D(
      pool_size=[2, 2]),
  tf.keras.layers.Dropout(0.35),
  tf.keras.layers.Flatten(),
  tfp.layers.DenseFlipout(
      128, kernel_divergence_fn=kl_divergence_function,
      activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.5),
  tfp.layers.DenseFlipout(
      NUM_CLASSES, kernel_divergence_fn=kl_divergence_function,
      activation=tf.nn.softmax)])

  
#   model = tf.keras.models.Sequential([
#       tfp.layers.Convolution2DFlipout(
#           6, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tf.keras.layers.MaxPooling2D(
#           pool_size=[2, 2], strides=[2, 2],
#           padding='SAME'),
#       tfp.layers.Convolution2DFlipout(
#           16, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tf.keras.layers.MaxPooling2D(
#           pool_size=[2, 2], strides=[2, 2],
#           padding='SAME'),
#       tfp.layers.Convolution2DFlipout(
#           120, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tf.keras.layers.Flatten(),
#       tfp.layers.DenseFlipout(
#           84, kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tfp.layers.DenseFlipout(
#           NUM_CLASSES, kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.softmax)
#   ])

  # Model compilation.
  optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
  # We use the categorical_crossentropy loss since the MNIST dataset contains
  # ten labels. The Keras API will then automatically add the
  # Kullback-Leibler divergence (contained on the individual layers of
  # the model), to the cross entropy loss, effectively
  # calcuating the (negated) Evidence Lower Bound Loss (ELBO)
  model.compile(optimizer, loss='categorical_crossentropy',
                metrics=['accuracy'], experimental_run_tf_function=False)
  return model


class MNISTSequence(tf.keras.utils.Sequence):
  """Produces a sequence of MNIST digits with labels."""

  def __init__(self, data=None, batch_size=128, fake_data_size=None):
    """Initializes the sequence.

    Args:
      data: Tuple of numpy `array` instances, the first representing images and
            the second labels.
      batch_size: Integer, number of elements in each training batch.
      fake_data_size: Optional integer number of fake datapoints to generate.
    """
    if data:
      images, labels = data
    else:
      images, labels = MNISTSequence.__generate_fake_data(
          num_images=fake_data_size, num_classes=NUM_CLASSES)
    self.images, self.labels = MNISTSequence.__preprocessing(
        images, labels)
    self.batch_size = batch_size

  @staticmethod
  def __generate_fake_data(num_images, num_classes):
    """Generates fake data in the shape of the MNIST dataset for unittest.

    Args:
      num_images: Integer, the number of fake images to be generated.
      num_classes: Integer, the number of classes to be generate.
    Returns:
      images: Numpy `array` representing the fake image data. The
              shape of the array will be (num_images, 28, 28).
      labels: Numpy `array` of integers, where each entry will be
              assigned a unique integer.
    """
    images = np.random.randint(low=0, high=256,
                               size=(num_images, IMAGE_SHAPE[0],
                                     IMAGE_SHAPE[1]))
    labels = np.random.randint(low=0, high=num_classes,
                               size=num_images)
    return images, labels

  @staticmethod
  def __preprocessing(images, labels):
    """Preprocesses image and labels data.

    Args:
      images: Numpy `array` representing the image data.
      labels: Numpy `array` representing the labels data (range 0-9).

    Returns:
      images: Numpy `array` representing the image data, normalized
              and expanded for convolutional network input.
      labels: Numpy `array` representing the labels data (range 0-9),
              as one-hot (categorical) values.
    """
#     images = 2 * (images / 255.) - 1.
#     images = images[..., tf.newaxis]

    labels = tf.keras.utils.to_categorical(labels)
    return images, labels

  def __len__(self):
    return int(tf.math.ceil(len(self.images) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.images[idx * self.batch_size: (idx + 1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
    return batch_x, batch_y





# model for spatial, regardless of channels

In [8]:

# train_set, heldout_set = tf.keras.datasets.mnist.load_data()
# train_seq = MNISTSequence(data=train_set, batch_size=batch_size)
# heldout_seq = MNISTSequence(data=heldout_set, batch_size=batch_size)


IMAGE_SHAPE = [52, 24, 2]

# NUM_CLASSES = 4

# learning_rate = 0.001

# num_epochs=10
batch_size=118


# Model / data parameters
total_count =len(all_normed)
# batch_size = 64
num_classes = 4

max_l,max_w = 52,24

input_shape = (2,max_l,max_w)


### train valid test: 80 20
p_train = 0.8
n_train = int(total_count*p_train )
# n_valid =  (total_count - n_train)



NUM_TRAIN_EXAMPLES = n_train
NUM_HELDOUT_EXAMPLES = total_count-n_train

x_train = all_normed[:n_train]
x_valid = all_normed[n_train:]
# x_test  = all_normed[n_train+n_valid:]

y_train = all_label[:n_train]
y_valid = all_label[n_train:]
# y_test  = all_label[n_train+n_valid:]

# # print(y_train.mean(),y_valid.mean(),y_test.mean())

# y_train = tf.keras.utils.to_categorical(y_train, num_classes)
# y_test  = tf.keras.utils.to_categorical(y_test, num_classes)
# y_valid = tf.keras.utils.to_categorical(y_valid, num_classes)




# x_train = x_train.reshape((x_train.shape[0],x_train.shape[2],x_train.shape[3],x_train.shape[1]))
# x_valid = x_valid.reshape((x_valid.shape[0],x_valid.shape[2],x_valid.shape[3],x_valid.shape[1]))
# x_test = x_test.reshape((x_test.shape[0],x_test.shape[2],x_test.shape[3],x_test.shape[1]))

In [20]:
kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) /  # pylint: disable=g-long-lambda
                        tf.cast(NUM_TRAIN_EXAMPLES, dtype=tf.float32))

# Define a LeNet-5 model using three convolutional (with max pooling)
# and two fully connected dense layers. We use the Flipout
# Monte Carlo estimator for these layers, which enables lower variance
# stochastic gradients than naive reparameterization.

#   model.add(Conv2D(32,
#                  activation='relu',
#                  input_shape=input_shape,
#                  kernel_size=(3, 3)))
# model.add(Conv2D(64, activation='relu',
#                  kernel_size=(3, 3)))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.35))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))  

model = tf.keras.models.Sequential([
  tfp.layers.Convolution2DFlipout(
      32, kernel_size=3, padding='SAME',
      kernel_divergence_fn=kl_divergence_function,
      activation=tf.nn.relu),
  tfp.layers.Convolution2DFlipout(
      64, kernel_size=3,
      activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D(
      pool_size=[2, 2]),
  tf.keras.layers.Dropout(0.35),
  tf.keras.layers.Flatten(),
  tfp.layers.DenseFlipout(
      128, kernel_divergence_fn=kl_divergence_function,
      activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.5),
  tfp.layers.DenseFlipout(
      NUM_CLASSES, kernel_divergence_fn=kl_divergence_function,
      activation=tf.nn.softmax)
])


#   model = tf.keras.models.Sequential([
#       tfp.layers.Convolution2DFlipout(
#           6, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tf.keras.layers.MaxPooling2D(
#           pool_size=[2, 2], strides=[2, 2],
#           padding='SAME'),
#       tfp.layers.Convolution2DFlipout(
#           16, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tf.keras.layers.MaxPooling2D(
#           pool_size=[2, 2], strides=[2, 2],
#           padding='SAME'),
#       tfp.layers.Convolution2DFlipout(
#           120, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tf.keras.layers.Flatten(),
#       tfp.layers.DenseFlipout(
#           84, kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.relu),
#       tfp.layers.DenseFlipout(
#           NUM_CLASSES, kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.softmax)
#   ])

# Model compilation.
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
# We use the categorical_crossentropy loss since the MNIST dataset contains
# ten labels. The Keras API will then automatically add the
# Kullback-Leibler divergence (contained on the individual layers of
# the model), to the cross entropy loss, effectively
# calcuating the (negated) Evidence Lower Bound Loss (ELBO)
model.compile(optimizer, loss='categorical_crossentropy',
            metrics=['accuracy'], experimental_run_tf_function=False)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_valid = tf.keras.utils.to_categorical(y_valid, num_classes)
model.fit(x_train, y_train, batch_size=batch_size,epochs=num_epochs, validation_data=(x_valid, y_valid))



Epoch 1/20
305/305 [==============================] - 160s 517ms/step - loss: 44016.7773 - accuracy: 0.3774 - val_loss: 41265.4961 - val_accuracy: 0.5811
Epoch 2/20
305/305 [==============================] - 166s 546ms/step - loss: 38591.1797 - accuracy: 0.6255 - val_loss: 35910.1953 - val_accuracy: 0.7378
Epoch 3/20
305/305 [==============================] - 168s 551ms/step - loss: 33300.8633 - accuracy: 0.6942 - val_loss: 30692.1895 - val_accuracy: 0.7631
Epoch 4/20
305/305 [==============================] - 203s 666ms/step - loss: 28170.9883 - accuracy: 0.7135 - val_loss: 25661.0605 - val_accuracy: 0.7657
Epoch 5/20
305/305 [==============================] - 170s 557ms/step - loss: 23259.5625 - accuracy: 0.7225 - val_loss: 20882.9883 - val_accuracy: 0.7761
Epoch 6/20
305/305 [==============================] - 174s 571ms/step - loss: 18640.7832 - accuracy: 0.7298 - val_loss: 16440.0586 - val_accuracy: 0.7886
Epoch 7/20
305/305 [==============================] - 170s 557ms/step - loss

In [16]:
# train_set, heldout_set = tf.keras.datasets.mnist.load_data()
# train_seq = MNISTSequence(data=train_set, batch_size=batch_size)
# heldout_seq = MNISTSequence(data=heldout_set, batch_size=batch_size)


train_set = (x_train, y_train)
heldout_set = (x_valid,y_valid)
train_seq = MNISTSequence(data=train_set, batch_size=118)
heldout_seq = MNISTSequence(data=heldout_set, batch_size=130)



model = create_model()
# TODO(b/149259388): understand why Keras does not automatically build the
# model correctly.
model.build(input_shape=[None, 52, 24, 2])

print(' ... Training convolutional neural network')
model.fit(x_train, y_train, batch_size=batch_size,epochs=num_epochs, validation_data=(x_valid, y_valid))


# for epoch in range(num_epochs):
#     epoch_accuracy, epoch_loss = [], []
# for step, (batch_x, batch_y) in enumerate(train_seq):
#   batch_loss, batch_accuracy = model.train_on_batch(
#       batch_x, batch_y)
#   epoch_accuracy.append(batch_accuracy)
#   epoch_loss.append(batch_loss)

#   if step % 100 == 0:
#     print('Epoch: {}, Batch index: {}, '
#           'Loss: {:.3f}, Accuracy: {:.3f}'.format(
#               epoch, step,
#               tf.reduce_mean(epoch_loss),
#               tf.reduce_mean(epoch_accuracy)))

#   if (step+1) % viz_steps == 0:
#     # Compute log prob of heldout set by averaging draws from the model:
#     # p(heldout | train) = int_model p(heldout|model) p(model|train)
#     #                   ~= 1/n * sum_{i=1}^n p(heldout | model_i)
#     # where model_i is a draw from the posterior p(model|train).
#     print(' ... Running monte carlo inference')
#     probs = tf.stack([model.predict(heldout_seq, verbose=1)
#                       for _ in range(num_monte_carlo)], axis=0)
#     mean_probs = tf.reduce_mean(probs, axis=0)
#     heldout_log_prob = tf.reduce_mean(tf.math.log(mean_probs))
#     print(' ... Held-out nats: {:.3f}'.format(heldout_log_prob))


#     names = [layer.name for layer in model.layers
#            if 'flipout' in layer.name]
#     qm_vals = [layer.kernel_posterior.mean().numpy()
#              for layer in model.layers
#              if 'flipout' in layer.name]
#     qs_vals = [layer.kernel_posterior.stddev().numpy()
#              for layer in model.layers
#              if 'flipout' in layer.name]
#     plot_weight_posteriors(names, qm_vals, qs_vals,
#                          fname=os.path.join(
#                              model_dir,
#                              'epoch{}_step{:05d}_weights.png'.format(
#                                  epoch, step)))
#     plot_heldout_prediction(heldout_seq.images, probs.numpy(),
#                           fname=os.path.join(
#                               model_dir,
#                               'epoch{}_step{}_pred.png'.format(
#                                   epoch, step)),
#                           title='mean heldout logprob {:.2f}'
#                           .format(heldout_log_prob))




 ... Training convolutional neural network
Epoch 1/20


ValueError: in user code:

    File "/Users/Mycroft/Documents/Documents/Documents/books/Harvard/Paulsson_Lab/Image_Processing/Barcoding/CNN_classifier/BNN_venv/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/Mycroft/Documents/Documents/Documents/books/Harvard/Paulsson_Lab/Image_Processing/Barcoding/CNN_classifier/BNN_venv/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/Mycroft/Documents/Documents/Documents/books/Harvard/Paulsson_Lab/Image_Processing/Barcoding/CNN_classifier/BNN_venv/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/Mycroft/Documents/Documents/Documents/books/Harvard/Paulsson_Lab/Image_Processing/Barcoding/CNN_classifier/BNN_venv/lib/python3.9/site-packages/ke

In [24]:
import os
import warnings

import seaborn as sns
from absl import app
from absl import flags
import matplotlib
matplotlib.use('Agg')
from matplotlib import figure  # pylint: disable=g-import-not-at-top
from matplotlib.backends import backend_agg
import numpy as np
import tensorflow.compat.v2 as tf
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
# import tensorflow_probability as tfp



tf.enable_v2_behavior()
warnings.simplefilter(action='ignore')

# tfd = tfp.distributions

In [26]:
import h5py
filename = "/Volumes/Suyang_T5/Paulsson_lab/Dyes/20210812/barcodes_normalized.h5"
with h5py.File(filename, "r") as f:
    BFP_normed = f.get('BFP_data')[:]
    BFP_label = f.get('BFP_label')[:]  # BFP RFP label second column for the other color
    RFP_normed = f.get('RFP_data')[:]
    RFP_label = f.get('RFP_label')[:]
    all_normed = f.get('all_data')[:]
    all_label = f.get('all_label')[:]


In [27]:
# reshape all data to be channel last

BFP_normed = BFP_normed.reshape((BFP_normed.shape[0], 52, 24,2))
RFP_normed = RFP_normed.reshape((RFP_normed.shape[0], 52, 24,2))
all_normed = all_normed.reshape((all_normed.shape[0], 52, 24,2))

# BFP_label = BFP_label.reshape((BFP_label.shape[0], ))
# RFP_label = RFP_label.reshape((RFP_label.shape[0], ))
all_label = all_label.reshape((all_label.shape[0], ))

In [28]:

# IMAGE_SHAPE = (52, 24, 2)
# # NUM_TRAIN_EXAMPLES = 60000
# # NUM_HELDOUT_EXAMPLES = 10000
# NUM_CLASSES = 4

# learning_rate = 0.001

# num_epochs=20
# batch_size=128
# data_dir = os.path.join("output","bayesian_neural_network/data")
# # Directory to put the model's fit.
# model_dir = os.path.join("output","bayesian_neural_network/")
# viz_steps = 400
# num_monte_carlo=50
# fake_data=False
nets = 1
model = [0] *nets
for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape =  (52, 24, 2)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(128, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(4, activation='softmax'))

    # COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
    model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    




In [29]:
# CREATE MORE IMAGES VIA DATA AUGMENTATION
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
# TRAIN NETWORKS
history = [0] * nets
epochs = 45
for j in range(nets):
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train, y_train, test_size = 0.1)
    history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),
        epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,  
        validation_data = (X_val2,Y_val2), callbacks=[annealer], verbose=0)
    print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
        j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

KeyboardInterrupt: 